In [ ]:
import json
import datetime
from collections import defaultdict

dayOfWeekReviews = defaultdict(dict)
with open('./data/yelp_academic_dataset_review.json', 'rb') as fread:
    for line in fread:
        jsonData = json.loads(line)
        date = datetime.datetime.strptime(jsonData['date'], '%Y-%m-%d')
        star = jsonData['stars']
        weekDay = date.weekday()
        
        weekDayStars = dayOfWeekReviews[weekDay]
        if star in weekDayStars:
            weekDayStars[star] += 1
        else:
            weekDayStars[star] = 1
            
        dayOfWeekReviews[weekDay] = weekDayStars

for k, v in dayOfWeekReviews.iteritems():
    print k, v


In [ ]:
for i in xrange(7):
    d = dayOfWeekReviews[i]
    print d[1], d[2], d[3], d[4], d[5], i

In [ ]:
import json
import datetime
from collections import defaultdict

businessTypes = defaultdict(int)
businessCountByCity = defaultdict(int)
businessTypeCountByCity = defaultdict(lambda : defaultdict(int))

with open('./data/yelp_academic_dataset_business.json', 'rb') as fread:
    for line in fread:
        jsonData = json.loads(line)
        city = jsonData['city']
        state = jsonData['state']
        categories = jsonData['categories']
        for cat in categories:
            businessTypes[cat] += 1
            businessTypeCountByCity[(city, state)][cat] += 1
            
        businessCountByCity[(city, state)] += 1
        

In [ ]:
print len(businessCountByCity)

In [ ]:
#for k, v in businessCountByCity.iteritems():
#    print k[0], '|', k[1], '|', v
import operator

print max(businessCountByCity.iteritems(), key=operator.itemgetter(1))[1]

In [ ]:
value = 1
dailyIncrease = .18/100
for i in xrange(36):
    value += value * dailyIncrease

print value

In [ ]:
reviewLengthByStar = defaultdict(list)
numOfStarsByUid = defaultdict(lambda : defaultdict(int))
with open('./data/yelp_academic_dataset_review.json', 'rb') as fread:
    for line in fread:
        jsonData = json.loads(line)
        star = jsonData['stars']
        weekDay = date.weekday()
        text = jsonData['text']
        userId = jsonData['user_id']
        reviewLengthByStar[star].append(len(text))
        numOfStarsByUid[userId][star] += 1


In [ ]:
import numpy as np
for star in reviewLengthByStar:
    reviewLengths = reviewLengthByStar[star]
    print star, np.mean(reviewLengths), np.std(reviewLengths), len(reviewLengths)

In [ ]:
for cat, count in businessTypes.iteritems():
    print cat, '|', count

In [ ]:
import toyplot.color
toyplot.color.Palette()

In [ ]:
import numpy as np

np.random.seed(1234)
observations = np.random.normal(size=(50, 50))

x = np.linspace(0, 1, len(observations))
y1 = np.min(observations, axis=1)
y2 = np.max(observations, axis=1)

canvas = toyplot.Canvas(width=400, height=300)
axes = canvas.axes()
mark = axes.fill(x, y1, y2)

In [1]:
from pymongo import MongoClient
import json
import datetime

In [2]:
client = MongoClient('localhost', 27017)
db = client.yelp_dataset
business = db.business

In [3]:
usBoundingBox = ((-124.848974, 24.396308), (-66.885444, 49.384358))
businessCursor = business.find({'categories':'Restaurants', 
                                'longitude': {'$lte': -66.885444}, 
                                'longitude': {'$gte': -124.848974}})

In [4]:
allBusiness = [b for b in businessCursor]

In [5]:
print len(allBusiness)
print allBusiness[12]

21892
{u'city': u'Homestead', u'review_count': 3, u'name': u"Eat'n Park Hospitality Group", u'neighborhoods': [u'Homestead'], u'open': True, u'business_id': u'sRqB6flj3GtTZIZJQxf_oA', u'full_address': u'285 Waterfront Dr E\nHomestead\nHomestead, PA 15120', u'hours': {}, u'state': u'PA', u'longitude': -79.9123428, u'stars': 2.5, u'latitude': 40.4116918, u'attributes': {u'Noise Level': u'very_loud', u'Parking': {u'garage': False, u'street': False, u'validated': False, u'lot': False, u'valet': False}, u'Alcohol': u'beer_and_wine', u'Attire': u'casual'}, u'_id': ObjectId('55d1dfe9fa8def3669b5fcf5'), u'type': u'business', u'categories': [u'Restaurants']}


In [6]:
reviews = db.reviews
tags = db.tags
tagCursor = tags.find()
tagged_reviews = [t['review_id'] for t in tagCursor]
print len(tagged_reviews), tagged_reviews[0]
businessIds = [b['business_id'] for b in allBusiness]
reviewCur = reviews.find({'business_id':{'$in':businessIds}})
usRestaurantReviews = [r for r in reviewCur]

537223 RdFot8wgSaCtPUatrAkiqA


In [9]:
print len(usRestaurantReviews)
print usRestaurantReviews[0]
allRestReviews = [r['review_id'] for r in usRestaurantReviews]
notTagged = list(set(allRestReviews) - set(tagged_reviews))
print len(notTagged)

990627
{u'votes': {u'funny': 1, u'useful': 1, u'cool': 1}, u'user_id': u'DBz7nWHg6tLK1JzLFOtU3A', u'review_id': u'RdFot8wgSaCtPUatrAkiqA', u'text': u'Great guys, great food. Plus right next to one of my favorite bars..', u'business_id': u'--5jkZ3-nUPZxUvtcbr8Uw', u'stars': 5, u'date': datetime.datetime(2011, 7, 31, 0, 0), u'_id': ObjectId('55d1de3bfa8def3669b0fb65'), u'type': u'review'}
453404


In [10]:
#notTaggedReviews = list(notTagged)
import nltk
from nltk.corpus import stopwords

stopwords = stopwords.words('english')

In [ ]:
import multiprocessing
import string

def worker(id, db, skip, limit):
    punctuations = list(string.punctuation)
    punctuations.append("''")
    reviewColls = db.reviews
    tagColls = db.tmptags
    batchSize = 100
    #for batch in xrange(0, limit, batchSize):
    if True:
        reviewCur = reviewColls.find({'review_id':{'$in':notTagged}}).skip(skip).limit(limit)
        bulkInsertData = []
        for review in reviewCur:
            words = []
            reviewText = review['text'].lower()
            businessId = review['business_id']
            userId = review['user_id']
            date = review['date']
            reviewId = review['review_id']
            stars = review['stars']
            sentences = nltk.sent_tokenize(reviewText)
            for sentence in sentences:
                tokens = nltk.word_tokenize(sentence)
                tokens = [t for t in tokens if t not in punctuations]
                text = [word for word in tokens if word not in stopwords]

                tagged_text = nltk.pos_tag(text)

                for word, tag in tagged_text:
                    words.append({"word": word, "pos": tag})

                insertData = {'business_id': businessId,
                              'user_id': userId,
                              'date': date,
                              'review_id': reviewId,
                              'stars': stars,
                              'words': words}
            #tagColls.insert(insertData)
        bulkInsertData.append(insertData)
        tagColls.insert_many(bulkInsertData)
#worker(1, db, 0, 0)
numWorkers = 2
#print notTagged[:50]
reviewCur = reviews.find({'review_id':{'$in':notTagged}})
totalReviews = reviewCur.count()
batchSize = totalReviews/numWorkers
extra = totalReviews % numWorkers

jobs = []
for i in xrange(numWorkers):
    left = 0
    if i == (numWorkers - 1):
        left = extra
    p = multiprocessing.Process(target=worker, args=((i + 1), db, i * batchSize, batchSize + left))
    jobs.append(p)
    p.start()

for j in jobs:
    j.join()

In [10]:
from nltk.stem.wordnet import WordNetLemmatizer

def create_corpus():
    lm = WordNetLemmatizer()
    

In [20]:
import json
import string

def create_json_array():
    punctuations = list(string.punctuation)
    punctuations.append("''")
    reviewColls = db.reviews
    tagColls = db.tmptags
    batchSize = 100
    bulkInsertData = []
    
    
    #for batch in xrange(0, limit, batchSize):
    if True:
        reviewCur = reviewColls.find({'review_id':{'$in':notTagged}})
        count = 0
        totalReviews = reviewCur.count()
        print "Total reviews", totalReviews
        percentComplete = 0
        tenPercentCount = totalReviews/10
        for review in reviewCur:
            count += 1
            words = []
            reviewText = review['text'].lower()
            businessId = review['business_id']
            userId = review['user_id']
            date = review['date']
            reviewId = review['review_id']
            stars = review['stars']
            sentences = nltk.sent_tokenize(reviewText)
            for sentence in sentences:
                tokens = nltk.word_tokenize(sentence)
                tokens = [t for t in tokens if t not in punctuations]
                text = [word for word in tokens if word not in stopwords]

                tagged_text = nltk.pos_tag(text)

                for word, tag in tagged_text:
                    words.append({"word": word, "pos": tag})

            insertData = {'business_id': businessId,
                              'user_id': userId,
                              'date': date,
                              'review_id': reviewId,
                              'stars': stars,
                              'words': words}
            #tagColls.insert(insertData)
            bulkInsertData.append(insertData)
            if count == tenPercentCount:
                percentComplete += 10
                print percentComplete,
                count = 0
        dump = json.dumps(bulkInsertData)
        return dump
        
dump = create_json_array()

Total reviews 453404
10 20 30

KeyboardInterrupt: 

In [27]:
punctuations = list(string.punctuation)
punctuations.append("''")
bulkInsertData = []
tagColls = db.tmptags
with open('./data/derivedData/tokenized.json', 'a') as fwrite:
    with open('./data/yelp_academic_dataset_review.json', 'rb') as fread:
        for line in fread:
            jsonData = json.loads(line)
            reviewId = jsonData['review_id']
            if reviewId in notTagged:
                words = []
                reviewText = jsonData['text'].lower()
                businessId = jsonData['business_id']
                userId = jsonData['user_id']
                date = jsonData['date']
                stars = jsonData['stars']
                sentences = nltk.sent_tokenize(reviewText)
                for sentence in sentences:
                    tokens = nltk.word_tokenize(sentence)
                    tokens = [t for t in tokens if t not in punctuations]
                    text = [word for word in tokens if word not in stopwords]

                    tagged_text = nltk.pos_tag(text)

                    for word, tag in tagged_text:
                        words.append({"word": word, "pos": tag})

                insertData = {'business_id': businessId,
                                  'user_id': userId,
                                  'date': date,
                                  'review_id': reviewId,
                                  'stars': stars,
                                  'words': words}
                json.dump(insertData, fwrite)
                fwrite.write('\n')
            #bulkInsertData.append(insertData)
#tagColls.insert_many(bulkInsertData)


KeyboardInterrupt: 